In [21]:
import pandas as pd
import time 
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from glob import glob
import IPython.display as ipd
from IPython.display import Audio
from itertools import cycle
from sklearn.model_selection import train_test_split
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import math
import pickle
import random
import operator
import os


In [23]:
path = os.getcwd()
print(path)

/Users/danielcontreras/LA Data Science Final project


In [24]:
#Right here we explore the audio files
audio_path = '/Users/danielcontreras/LA Data Science Final project/data/genres_original'
images_path = '/Users/danielcontreras/LA Data Science Final project/data/images_original'
metadata_path = '/Users/danielcontreras/LA Data Science Final project/data/features_30_sec.csv'
metadata = pd.read_csv(metadata_path)

#Audio(audio_path+"/hiphop/hiphop.00051.wav")
#Audio(audio_path+"/hiphop/hiphop.00050.wav")
#Audio(audio_path+"/pop/pop.00060.wav")
#Audio(audio_path+"/jazz/jazz.00054.wav")
Audio(audio_path+"/reggae/reggae.00083.wav")
#second movement of Ravel's string quartet is from file no. 34 under classical folder
#file 50 from the hiphop folder is dope af


In [25]:
#define a function to get distance between feature vectors and find neighbors. We use the Mahalanobis distance
#Credit for some of the code goes to https://www.kaggle.com/code/rxsraghavagrawal/music-genre-classification-using-knn-begineers/notebook
def distance(vector1, vector2, k):
    distance = 0
    mm1 = vector1[0]
    cm1 = vector1[1]
    mm2 = vector2[0]
    cm2 = vector2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

#In the next two functions we implement the K nearest neighbors algorithm
def getNeighbors(trainingset, instance, k):
    distances = []
    for i in range(len(trainingset)):
        dist = distance(trainingset[i], instance, k) + distance(instance, trainingset[i],k)
        distances.append((trainingset[i][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

def nearestclass(neighbors):
    voteCount = {}
    
    for i in range(len(neighbors)):
        if neighbors[i] in voteCount:
            voteCount[neighbors[i]] += 1
        else:
            voteCount[neighbors[i]] = 1
    
    updatedVoteCount = sorted(voteCount.items(), key=lambda count: count[1], reverse=True)
    
    return updatedVoteCount[0][0]

def AccuracyMeasure(testData, predictions):
    correct = 0
    
    for i in range(len(testData)):
        if testData[i] == predictions[i]:
            correct += 1
    
    accuracy = 100 * (correct/len(testData))
    
    return accuracy



In [26]:
#Right here we populate a newly created file with the new feature vectors of each audio signal (song samples)
#from the GTZAN dataset
directory = audio_path
f = open("processedData", "wb")
i = 0
for folder in os.listdir(directory):
    #print(folder)
    i += 1
    if i == 11:
        break
    elif folder == '.DS_Store':
        i -= 1
        continue
    for file in os.listdir(directory+"/"+folder):
        #print(file)
        try:
            (rate, sig) = wav.read(directory+"/"+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception: ", e, 'in folder: ', folder, ' filename: ', file)
#f.close()

#Right here we create a list of the processed data so that we can perform the train/test splitting
dataset = []

def loadDataset():
    with open('processedData','rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
                
loadDataset()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported. in folder:  jazz  filename:  jazz.00054.wav


In [27]:
X = []
y = []
for i in range(len(dataset)):
    X.append(dataset[i][0:2])
    y.append(dataset[i][2])
#We use scikitLearn functionality to split training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#Here we populate the training and testing datasets as lists with tuples of size 3
'''
updatedDataset = []
for i in range(len(X)):
    updatedDataset.append((X[i][0], X[i][1], y[i])
'''

testingDataset = []
for i in range(len(X_test)):
    testingDataset.append((X_test[i][0], X_test[i][1], y_test[i]))

trainingDataset = []
for i in range(len(X_test)):
    trainingDataset.append((X_train[i][0], X_train[i][1], y_train[i]))

In [16]:
length = len(X_test)
predictions = []
for x in range(length):
    predictions.append(nearestclass(getNeighbors(dataset, X_test[x], 5)))

accuracy1 = AccuracyMeasure(y_test, predictions)

print(accuracy1)



84.5


In [28]:
results = {}
i = 1
for folder in os.listdir(directory):
    if folder == '.DS_Store':
        continue
    else:
        results[i] = folder
        i += 1

In [34]:
#(rate, sig) = wav.read(audio_path+"/classical/classical.00050.wav")
(rate, sig) = wav.read(audio_path+"/classical/classical.00083.wav")
mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance)

In [33]:
pred = nearestclass(getNeighbors(dataset, feature, 5))
print(results[pred])

classical


In [40]:
#Function to check the accuracy of the algorithm per genre
def accuracyGenre(genre, comparison):
    i = 0
    #In this loop we process the audio data for each file in the folder of the desired genre
    for file in os.listdir(directory+'/'+genre):
        (rate, sig) = wav.read(directory+'/'+genre+'/'+file)
        mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix, covariance)
        pred = nearestclass(getNeighbors(dataset, feature, 5)) #right here we get the prediction using KNN for the current file
        if results[pred] == comparison: #This if statement serves to check if the prediction matches the genre
            i += 1                 #the audio belongs to, i.e the genre we used as input for the function.
        else:
            continue
    return i/len(os.listdir(directory+'/'+genre))*100

print(accuracyGenre('rock', 'blues'))

3.0
